In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import scipy
import math
import random
import cvxopt
from cvxopt import matrix, solvers
from scipy.optimize import minimize
import statistics
%matplotlib inline

In [2]:
tickers_list = ['AAPL', 'TSLA', 'IBM', 'NFLX', 'FB', 'AXP', 'COKE', 'GOOG', 'AMZN', 'WMT']
data = yf.download(tickers_list,'2016-1-1')['Adj Close']

[*********************100%***********************]  10 of 10 completed


In [3]:
returns = data.pct_change()
returns.head()

,AAPL,AMZN,AXP,COKE,FB,GOOG,IBM,NFLX,TSLA,WMT
Date,,,,,,,,,,
2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-04,0.000855,-0.057554,-0.028181,-0.040874,-0.023314,-0.022454,-0.012135,-0.038643,-0.069164,0.002610
2016-01-05,-0.025059,-0.005024,-0.015387,0.009654,0.004989,0.000998,-0.000735,-0.020917,0.000090,0.023755
2016-01-06,-0.019570,-0.001799,-0.027769,0.018162,0.002336,0.001400,-0.005006,0.093071,-0.019648,0.010013
2016-01-07,-0.042204,-0.039058,-0.009003,-0.014004,-0.049043,-0.023170,-0.017090,-0.026513,-0.015477,0.023289


In [4]:
ui = returns.iloc[1:,:].mean()
# rit = ((ui+1).product())**(1/len(ui)) - 1
ui

AAPL    0.001412
AMZN    0.001265
AXP     0.000954
COKE    0.000813
FB      0.000984
GOOG    0.001021
IBM     0.000254
NFLX    0.001514
TSLA    0.002653
WMT     0.000781
dtype: float64

In [5]:
# returns = data.shift(1)/data
logReturns = returns

#calculate geometric mean 
ui = ui
#compute the co-variance matrix
Sigma = logReturns.cov()
pBar = logReturns.mean()


median = statistics.median(ui)
rMin = median

def riskFunction(w):
    return np.dot(w.T, np.dot(Sigma,w))

w0 = [1/len(tickers_list)]*(len(tickers_list))
bounds = ((0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1))
#bounds = ((0,1),(0,1),(0,1),(0,1))
def checkMinimumReturn(w):
    RHS = rMin - np.sum(pBar*w)
    return RHS

def CheckSumToOne(w):
    return np.sum(w)-1

constraints  = ({'type':'eq', 'fun' : checkMinimumReturn},
                {'type':'eq', 'fun' : CheckSumToOne})

w_opt = minimize(riskFunction, w0, method="SLSQP", bounds=bounds, constraints=constraints)
w_scipy = w_opt.x


risk_list = []
return_list = []
risk_list.append(riskFunction(252*w_scipy))
return_list.append(25200*median)


w = []
for i in range(len(w_scipy)):
    w.append(round(w_scipy[i], 5))
    
weights_avg = pd.DataFrame(returns.columns.T)
weights_avg.columns = ['Stock']
weights_avg['Weights_0'] = w
weights_avg

,Stock,Weights_0
0,AAPL,0.08885
1,AMZN,0.09551
2,AXP,0.10952
3,COKE,0.11590
4,FB,0.10811
5,GOOG,0.10651
6,IBM,0.14109
7,NFLX,0.08425
8,TSLA,0.03280
9,WMT,0.11746


In [6]:
cnt = 1
for i in range(4):
    u = []
    for i in ui:
        r = random.uniform(0.95, 1.05)
        u.append(i*r)
    ui = u
    
    #compute the co-variance matrix
    Sigma = logReturns.cov()
    pBar = logReturns.mean()


    median = statistics.median(ui)
    rMin = median

    def riskFunction(w):
        return np.dot(w.T, np.dot(Sigma,w))

    w0 = [1/len(tickers_list)]*(len(tickers_list))
    bounds = ((0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1))
    #bounds = ((0,1),(0,1),(0,1),(0,1))
    def checkMinimumReturn(w):
        RHS = rMin - np.sum(pBar*w)
        return RHS

    def CheckSumToOne(w):
        return np.sum(w)-1

    constraints  = ({'type':'eq', 'fun' : checkMinimumReturn},
                    {'type':'eq', 'fun' : CheckSumToOne})

    w_opt = minimize(riskFunction, w0, method="SLSQP", bounds=bounds, constraints=constraints)
    w_scipy = w_opt.x
    risk_list.append(riskFunction(252*w_scipy))
    return_list.append(25200*median)
    
    
    
    w = []
    for i in range(len(w_scipy)):
        w.append(round(w_scipy[i], 5))

    c = "Weights_" + str(cnt)
    weights_avg[c] = w
    cnt = cnt+1

In [7]:
weights_avg['Weights_Mean'] = weights_avg.mean(axis=1)
# print(weights_avg.to_latex(index =False))
weights_avg

,Stock,Weights_0,Weights_1,Weights_2,Weights_3,Weights_4,Weights_Mean
0,AAPL,0.08885,0.08903,0.09091,0.09135,0.09142,0.090312
1,AMZN,0.09551,0.09558,0.09634,0.09651,0.09654,0.096096
2,AXP,0.10952,0.10937,0.10777,0.10739,0.10733,0.108276
3,COKE,0.11590,0.11565,0.11298,0.11236,0.11225,0.113828
4,FB,0.10811,0.10798,0.10661,0.10628,0.10623,0.107042
5,GOOG,0.10651,0.10640,0.10531,0.10505,0.10501,0.105656
6,IBM,0.14109,0.14043,0.13352,0.13189,0.13162,0.135710
7,NFLX,0.08425,0.08450,0.08714,0.08777,0.08787,0.086306
8,TSLA,0.03280,0.03387,0.04516,0.04781,0.04826,0.041580
9,WMT,0.11746,0.11718,0.11427,0.11358,0.11347,0.115192


In [8]:
return_list.append(np.mean(return_list))
risk_list.append(np.mean(risk_list))
r_r = [weights_avg.columns[1:7], risk_list, return_list]
rr = pd.DataFrame(r_r).T
rr.columns = ['Weighting','Risk %', 'Return %']
rr

,Weighting,Risk %,Return %
0,Weights_0,10.003428,25.259477
1,Weights_1,10.018545,25.325215
2,Weights_2,10.189239,26.014184
3,Weights_3,10.232681,26.176396
4,Weights_4,10.240187,26.203962
5,Weights_Mean,10.136816,25.795847
